### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности.

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [90]:
# считаем данные
import pandas as pd
data = pd.read_csv("ner_data_train.csv")

In [91]:
data.head(5)

,video_info,entities
0,"<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Русский тизер=трейлер (2021) <ОПИСАНИЕ:>Лучший Telegram канал о кино <LINK> Сотрудничество <LINK> Дата выхода 26 августа 2021 Оригинальное название: OSS 117: Alerte rouge en Afrique noire Страна: Франция Режиссер: Николя Бедос Жанр: боевик, комедия В ...","{""label"":""локация""\,""offset"":26\,""length"":6\,""segment"":""Африки""},{""label"":""Дата""\,""offset"":155\,""length"":15\,""segment"":""26 августа 2021""},{""label"":""название проекта""\,""offset"":194\,""length"":38\,""segment"":""OSS 117: Alerte rouge en Afrique noire""},{""label"":""локация""\,""offset"":241\,""length"":7\,""seg..."
1,"<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 VK56 VK56DE VK56=DE Коленный Вал Infiniti QX56= 5.6 стандартный <ОПИСАНИЕ:><LINK> kolenval infiniti iks.html Коленвал Инфинити Ку икс 56​ Коленный Вал Infiniti QX56​ Модель Двигателя: VK56, VK56DE, VK56 DE Рабочий объем двигателя: 5.6 Мощность в л.с. ...","{""label"":""организация""\,""offset"":196\,""length"":21\,""segment"":""Infiniti QX56​ Модель""}"
2,"<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Человека [Короткометражный фильм] #серыйчеловек, #короткометражныйфильм <ОПИСАНИЕ:>ВЫЗОВ ДЕМОНА Вызвал Серого Человека [Короткометражный фильм] #серыйчеловек, #короткометражныйфильм Спасибо за просмотр. В качестве оплаты с вас лайк и подписка) Ссылки на м...","{""label"":""название проекта""\,""offset"":12\,""length"":37\,""segment"":""ВЫЗОВ ДЕМОНА = Вызвал Серого Человека""},{""label"":""название проекта""\,""offset"":124\,""length"":35\,""segment"":""ВЫЗОВ ДЕМОНА Вызвал Серого Человека""}"
3,"<НАЗВАНИЕ:> Довоенная немецкая кирха в Калининградской области. Посетили вместе с #Заброшки39! <ОПИСАНИЕ:>Хочешь новых эмоций, побывать в необычных местах, тогда присоединяйся к нам <AT>! Автомобильные путешествия по заброшенным старинным сооружениям. Самые необычные места Калининградской област...","{""label"":""не найдено""\,""offset"":162\,""length"":13\,""segment"":""присоединяйся""}"
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомотив засудили? Стоило ли отменять гол? <ОПИСАНИЕ:>Спартак #Локомотив #обзор #обзорматча #всеголы #Локо #ЛокомотивМосква #голПруцева #ЛокомотивСпартакобзор #ЛокомотивСпартакполныйматч #Кубок #КубокРоссии #ЛокомотивСпартаксудья #отмененныйгол #ЛокомотивСп...","{""label"":""команда""\,""offset"":13\,""length"":8\,""segment"":""Спартаку""},{""label"":""команда""\,""offset"":38\,""length"":9\,""segment"":""Локомотив""},{""label"":""команда""\,""offset"":93\,""length"":7\,""segment"":""Спартак""},{""label"":""команда""\,""offset"":102\,""length"":9\,""segment"":""Локомотив""},{""label"":""команда""\,""offse..."


In [92]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать,
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [93]:
df.head(3)

,video_info,entities
0,"<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Русский тизер=трейлер (2021) <ОПИСАНИЕ:>Лучший Telegram канал о кино <LINK> Сотрудничество <LINK> Дата выхода 26 августа 2021 Оригинальное название: OSS 117: Alerte rouge en Afrique noire Страна: Франция Режиссер: Николя Бедос Жанр: боевик, комедия В ...","[{'label': 'локация', 'offset': 26, 'length': 6, 'segment': 'Африки'}, {'label': 'Дата', 'offset': 155, 'length': 15, 'segment': '26 августа 2021'}, {'label': 'название проекта', 'offset': 194, 'length': 38, 'segment': 'OSS 117: Alerte rouge en Afrique noire'}, {'label': 'локация', 'offset': 241..."
1,"<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 VK56 VK56DE VK56=DE Коленный Вал Infiniti QX56= 5.6 стандартный <ОПИСАНИЕ:><LINK> kolenval infiniti iks.html Коленвал Инфинити Ку икс 56​ Коленный Вал Infiniti QX56​ Модель Двигателя: VK56, VK56DE, VK56 DE Рабочий объем двигателя: 5.6 Мощность в л.с. ...","[{'label': 'организация', 'offset': 196, 'length': 21, 'segment': 'Infiniti QX56​ Модель'}]"
2,"<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Человека [Короткометражный фильм] #серыйчеловек, #короткометражныйфильм <ОПИСАНИЕ:>ВЫЗОВ ДЕМОНА Вызвал Серого Человека [Короткометражный фильм] #серыйчеловек, #короткометражныйфильм Спасибо за просмотр. В качестве оплаты с вас лайк и подписка) Ссылки на м...","[{'label': 'название проекта', 'offset': 12, 'length': 37, 'segment': 'ВЫЗОВ ДЕМОНА = Вызвал Серого Человека'}, {'label': 'название проекта', 'offset': 124, 'length': 35, 'segment': 'ВЫЗОВ ДЕМОНА Вызвал Серого Человека'}]"


#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

In [3]:
!pip install razdel
!pip install transformers[torch]
!pip install datasets seqeval
!pip install corus razdel
!pip install torch==2.0.0
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=a371b13945046420c0d087f9a635129cd5ee2fdded92375639d9a1fab2d

In [94]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах

    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [95]:
print(extract_labels(df.iloc[0]))

{'tokens': ['<', 'НАЗВАНИЕ', ':', '>', 'Агент', '117', ':', 'Из', 'Африки', 'с', 'любовью', '—', 'Русский', 'тизер', '=', 'трейлер', '(', '2021', ')', '<', 'ОПИСАНИЕ', ':', '>', 'Лучший', 'Telegram', 'канал', 'о', 'кино', '<', 'LINK', '>', 'Сотрудничество', '<', 'LINK', '>', 'Дата', 'выхода', '26', 'августа', '2021', 'Оригинальное', 'название', ':', 'OSS', '117', ':', 'Alerte', 'rouge', 'en', 'Afrique', 'noire', 'Страна', ':', 'Франция', 'Режиссер', ':', 'Николя', 'Бедос', 'Жанр', ':', 'боевик', ',', 'комедия', 'В', 'главных', 'ролях', ':', 'Жан', 'Дюжарден', ',', 'Пьер', 'Нинэ', ',', 'Мелоди', 'Каста', ',', 'Наташа', 'Линдинжер', ',', 'Владимир', 'Иорданов', ',', 'Фату', 'Н', '’', 'Диайе', ',', 'Пол', 'Уайт', 'Мир', 'изменился', '.', 'Он', 'нет', '.', 'Судьба', 'заносит', 'легендарного', 'Агента', '117', 'в', 'Африку', ',', 'где', 'горячее', 'пустыни', 'только', 'женщины', '.', 'Вооруженный', 'неиссякаемой', 'уверенностью', 'в', 'себе', 'и', 'убийственной', 'харизмой', ',', 'он', 'мож

### Обработаем датасет и разобьем на трейн и тест

In [96]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [97]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
838,"[<, НАЗВАНИЕ, :, >, Интервью, с, Александром, Татаринчиком, ,, "", Пиро, =, шоу, "", ,, команда, России, ,, "", Ростех, =, 2016, "", ,, 24.07.2016, <, ОПИСАНИЕ, :, >, Интервью, с, Александром, Татаринчиком, ,, "", Пиро, шоу, "", ,, команда, России, ,, "", Ростех, "", ,, 24.07.2016, Продажа, пиротехники,...","[O, O, O, O, O, O, B-персона, I-персона, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
977,"[<, НАЗВАНИЕ, :, >, Борьба, на, практике, !, Отработка, и, закрепление, пройденного, материала, <, ОПИСАНИЕ, :, >, На, борцовской, тренировке, ,, после, прохождения, материала, и, отработки, ,, переходят, к, практике, ., Помощь, на, развитие, канала, 4276, 1609, 1349, 6650, Бусти, <, LINK, >, Ро...","[O, O, O, O, B-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O..."
3984,"[<, НАЗВАНИЕ, :, >, Веста, СВ, Кросс, (, 2022, ), :, ТОП, =, 10, бесячих, мелочей, !, И, ,, конечно, ,, про, масложор, ., <, ОПИСАНИЕ, :, >, Когда, проходит, эйфория, от, покупки, нового, автомобиля, ,, начинаешь, обращать, внимание, на, отдельные, недоработки, ., Некоторые, из, них, как, ложечк...","[O, O, O, O, B-модель, I-модель, I-модель, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O..."


#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [98]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [99]:
from datasets import Dataset, DatasetDict

In [100]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5137
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1285
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [101]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

model_checkpoint = "dslim/bert-base-NER"
batch_size = 32

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='gpu')

In [102]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'Московский', 'международный', 'фестиваль', 'мира', '=', '89', 'и', 'Стас', 'Намин', '.', '«', 'Главный', 'день', '»', '<', 'ОПИСАНИЕ', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центральном', 'стадионе', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'международный', 'рок', 'фестиваль', '.', 'Концерт', 'собрал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'молодежь', 'тогда', 'увидела', 'и', 'услышала', 'признанных', 'мировых', 'кумиров', ':', 'Scorpions', ',', 'Bon', 'Jovi', ',', 'Ozzy', 'Osbourne', ',', 'Cinderella', ',', 'Motley', 'Crue', '.', 'Приезд', 'западных', 'звезд', 'казался', 'ожившей', 'фантастикой', '.', 'Фестиваль', 'стал', 'не', 'только', 'музыкальной', ',', 'но', 'и', 'важной', 'политической', 'акцией', '.', 'Вся', 'мировая', 'пресса', 'писала', 'о', 'том', ',', 'что', 'СССР', 'и', 'США', ',', 'долгое', 'время', 'находившиеся', 'в', 'состоянии', '"', 'холодной', 'войны', '"', ',', 'наконец', 'то', 'с

In [103]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'Н', '##АЗ', '##В', '##АН', '##И', '##Е', ':', '>', 'Московский', 'между', '##народный', 'фестиваль', 'мира', '=', '89', 'и', 'С', '##тас', 'На', '##мин', '.', '«', 'Главный', 'день', '»', '<', 'О', '##П', '##И', '##С', '##АН', '##И', '##Е', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центр', '##альном', 'стадион', '##е', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'между', '##народный', 'рок', 'фестиваль', '.', 'К', '##он', '##цер', '##т', 'со', '##брал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'м', '##оло', '##де', '##ж', '##ь', 'тогда', 'у', '##вид', '##ела', 'и', 'у', '##сл', '##ыш', '##ала', 'признан', '##ных', 'мир', '##овых', 'к', '##уми', '##ров', ':', 'Sc', '##orp', '##ions', ',', 'Bon', 'Jovi', ',', 'Oz', '##zy', 'Os', '##bour', '##ne', ',', 'Ci', '##nder', '##ella', ',', 'Mot', '##ley', 'C', '##rue', '.', 'При', '##езд', 'запад', '##ных', 'зв', '##езд', 'каз', '##ался', 'о', '##жив', '##шей', 'ф', '##ант', '##аст'

In [104]:
len(example["tags"]), len(tokenized_input["input_ids"])

(170, 274)

In [105]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 11, 12, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 19, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 43, 43, 43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 51, 51, 52, 53, 53, 53, 54, 55, 55, 55, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 62, 63, 64, 65, 65, 66, 66, 66, 67, 68, 68, 68, 69, 70, 70, 71, 71, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 77, 78, 78, 78, 78, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 88, 89, 90, 90, 90, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 104, 105, 106, 106, 106, 107, 108, 109, 110, 110, 110, 111, 112, 113, 114, 115, 116, 117, 117, 117, 118, 118, 118, 118, 119, 120, 121, 121, 122, 123, 124, 124, 125, 125, 126, 126, 127, 127, 127, 128, 128, 129, 130, 130, 130, 130, 131, 132, 132, 133, 133, 134, 135, 136, 137, 137, 137, 138, 139, 139, 139, 140, 141, 142, 143, 144, 144, 144, 

In [106]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

274 274


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [107]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [108]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 32, 293, 16218, 8117, 20795, 8759, 14386, 30, 34, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 32, 294, 3932, 8759, 3330, 20795, 8759, 14386, 30, 34, 294, 18147, 5972, 2386, 1928, 733, 22970, 613, 17565, 29484, 15479, 12122, 18398, 13334, 1, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 18, 282, 3200, 860, 20727, 12029, 4375, 29463, 2013, 2262, 18, 294, 15735, 29137, 320, 18029, 18398, 24952, 753, 705, 11316, 4307, 329, 22067, 12427, 283, 4297, 292, 29172, 3330, 1928, 705, 13790, 294, 22797, 1768, 10203, 761, 13951, 3243, 2535, 2641, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [109]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [110]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers.optimization import Adafactor, AdafactorSchedule
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [111]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [112]:
metric = load_metric("seqeval")

In [113]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'бренд': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [22]:

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none'

)

In [23]:
# что мы видим без дообучения модели
import numpy as np
from torch import optim
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
# optimizer = Adafactor(
#             model.parameters(),
#             scale_parameter=True, # If True, learning rate is scaled by root mean square
        # )
# lr_scheduler = AdafactorSchedule(optimizer)
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # optimizers=(optimizer, lr_scheduler)
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.1704518795013428,
 'eval_precision': 0.003226199743029439,
 'eval_recall': 0.03874310170064196,
 'eval_f1': 0.005956400533305629,
 'eval_accuracy': 0.09838389777213814,
 'eval_runtime': 18.001,
 'eval_samples_per_second': 71.385,
 'eval_steps_per_second': 2.278}

In [24]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [25]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [26]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.637167,0.116223,0.002703,0.005283,0.869990
2,No log,0.570917,0.198484,0.032436,0.055760,0.877953
3,No log,0.548743,0.201849,0.051639,0.082238,0.878660
4,0.741200,0.536430,0.226988,0.055130,0.088714,0.879769
5,0.741200,0.532779,0.238337,0.060705,0.096764,0.880234


TrainOutput(global_step=805, training_loss=0.6653991320118401, metrics={'train_runtime': 212.5496, 'train_samples_per_second': 120.842, 'train_steps_per_second': 3.787, 'total_flos': 148874705043372.0, 'train_loss': 0.6653991320118401, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5358530282974243,
 'eval_precision': 0.21351351351351353,
 'eval_recall': 0.04893569095618876,
 'eval_f1': 0.07962250320689022,
 'eval_accuracy': 0.879237012201702,
 'eval_runtime': 8.7948,
 'eval_samples_per_second': 146.108,
 'eval_steps_per_second': 4.662,
 'epoch': 5.0}

In [114]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Дата': {'precision': 0.414983164983165,
  'recall': 0.41083333333333333,
  'f1': 0.4128978224455611,
  'number': 1200},
 'бренд': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 866},
 'вид спорта': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 251},
 'видеоигра': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 746},
 'команда': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 412},
 'лига': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 141},
 'локация': {'precision': 0.42857142857142855,
  'recall': 0.008985024958402662,
  'f1': 0.017601043024771838,
  'number': 3005},
 'модель': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 488},
 'название проекта': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 3270},
 'организация': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1631},
 'персона': {'precision': 0.17442950922163175,
  'recall': 0.10453353315848632,
  'f1': 0.13072507906758815,
  'number': 5338},
 'сезон': {'prec

In [115]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [116]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-Дата,B-бренд,B-вид спорта,B-видеоигра,B-команда,B-лига,B-локация,B-модель,B-название проекта,...,I-видеоигра,I-команда,I-лига,I-локация,I-модель,I-название проекта,I-организация,I-персона,I-сезон,I-серия
O,231314,342,0,0,0,0,0,9,0,0,...,0,0,0,0,0,22,0,1702,0,0
B-Дата,609,577,0,0,0,0,0,1,0,0,...,0,0,0,0,0,2,0,10,0,0
B-бренд,837,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,0,15,0,0
B-вид спорта,237,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12,0,0
B-видеоигра,729,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,12,0,5,0,0
B-команда,366,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,41,0,0
B-лига,136,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
B-локация,2359,10,0,0,0,0,0,32,0,1,...,0,0,0,0,0,1,0,520,0,0
B-модель,444,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,7,0,33,0,0
B-название проекта,3146,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,104,0,0


In [117]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

### Посмотрим на результаты

In [119]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']
text = ["Как", "сделать"]

In [33]:
!pip install transformers

In [120]:
import torch
from transformers import pipeline
import re
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

def predict_ner(text, tokenizer, model, pipe, verbose=True):

    dismissed_token = re.compile(r'\xad+|\u200b+')
    text = [re.sub(dismissed_token, '[UNK]', tok) for tok in text]

    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    words = tokens.words()

    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)

    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    labels = []
    prev=words[1] # это всегда ноль - первое слово
    labels = [label_list[indices[1]]]
    for word, tag in zip(words[1:-1], indices[1:-1]):
        if word != prev:
            labels.append(label_list[tag])
            prev=word
    return labels

In [121]:
predict_ner(text, tokenizer, model, pipe)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(


['I-серия', 'B-сезон']

In [ ]:
print(type(ner_test[2]['tokens']))

<class 'list'>


### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [122]:
submission = pd.DataFrame(columns=[['video_info', 'entities_prediction', "size_data", "size_prediction"]])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(text, tokenizer, model, pipe, submission):
    video_info = []
    for i, elem in tqdm(enumerate(ner_test)):
        labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
        video_info.append(elem['tokens'])
        #submission.loc[i, 'video_info'] = elem[0]
        entities_prediction =  [[label] for label in labels]
        submission.loc[i, 'entities_prediction'] = entities_prediction
    return submission,video_info

In [123]:
data = pd.read_csv("ner_data_test.csv")
result, video_info = sample_submission(data, tokenizer, model, pipe, submission)
result.to_csv('submission.csv', index=False)


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, 

In [124]:
import pandas as pd
csv_input = pd.read_csv('submission.csv')
csv_input['video_info'] = video_info
csv_input.to_csv('submission.csv', index=False)

In [125]:
csv_input

,video_info,entities_prediction,size_data,size_prediction
0,"[<, НАЗВАНИЕ, :, >, Самые, красивые, места, Боливии, ., Короткий, видеообзор, ., Достопримечательности, Боливии, ., Слайд, =, Шоу, ., <, ОПИСАНИЕ, :, >, Самые, красивые, места, Боливии, ., Короткий, видеообзор, ., Достопримечательности, Боливии, ., Слайд, Шоу, ., #, боливия, #, достопримечательн...","['B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-Дата', 'I-лига', 'B-Дата', 'B-Дата', 'B-персона', 'I-серия', 'I-лига', 'B-команда', 'B-Дата', 'B-Дата', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-название проекта...",NaN,NaN
1,"[<, НАЗВАНИЕ, :, >, ЭТО, ЖЕСТЬ, !, ОН, ХОТЕЛ, УБИТЬ, МЕНЯ, !, РЕЙД, БАЗЫ, С, ГРОМИЛОЙ, =, Last, Day, on, Earth, :, Survival, <, ОПИСАНИЕ, :, >, ЭТО, ЖЕСТЬ, !, ОН, ХОТЕЛ, УБИТЬ, МЕНЯ, !, РЕЙД, БАЗЫ, С, ГРОМИЛОЙ, Спасибо, за, Лайк, и, Подписку, ✅, Все, видео, LDOE, ►, <, LINK, >, SVVUGN_b, 7, gKVR...","['B-персона', 'B-персона', 'B-персона', 'B-персона', 'I-лига', 'I-лига', 'I-бренд', 'B-Дата', 'I-лига', 'B-персона', 'B-персона', 'I-бренд', 'B-персона', 'B-персона', 'B-персона', 'B-вид спорта', 'B-персона', 'B-Дата', 'B-персона', 'B-персона', 'I-локация', 'B-персона', 'B-персона', 'B-персона',...",NaN,NaN
2,"[<, НАЗВАНИЕ, :, >, Оранжевый, —, хит, сезона, /, Orange, Is, the, New, Black, (, 6, сезон, ), Русский, трейлер, (, субтитры, ), <, ОПИСАНИЕ, :, >, Премьера, :, 27, июля, 2018, г, ., Канал, :, Netflix, ., Жанр, :, драма, ., В, ролях, :, Тейлор, Шиллинг, ,, Узо, Адуба, ,, Лора, Припон, ,, Наташа,...","['B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-Дата', 'I-сезон', 'I-бренд', 'I-сезон', 'I-бренд', 'B-Дата', 'I-бренд', 'I-название проекта', 'B-Дата', 'I-бренд', 'I-бренд', 'I-команда', 'I-бренд', 'I-локация', 'B-бренд', 'B-вид спорта', 'I-команда', 'I-серия', 'I-локация', 'B-персона', ...",NaN,NaN
3,"[<, НАЗВАНИЕ, :, >, Голодные, игры, 3, :, Сойка, =, пересмешница, ., Часть, 1, (, 2014, ), —, русский, трейлер, <, ОПИСАНИЕ, :, >, Переходите, на, наш, канал, "", ТРЕЙЛЕРЫ, |, Ужасы, и, Триллеры, "", ➠, <, LINK, >, /, Там, вы, найдете, множество, интересных, трейлеров, к, фильмам, в, жанре, ужасы,...","['B-персона', 'B-персона', 'B-персона', 'B-персона', 'I-бренд', 'I-бренд', 'I-бренд', 'I-бренд', 'B-Дата', 'B-название проекта', 'B-Дата', 'I-персона', 'B-название проекта', 'I-серия', 'I-бренд', 'I-локация', 'I-локация', 'I-серия', 'B-название проекта', 'B-вид спорта', 'B-персона', 'B-персона',...",NaN,NaN
4,"[<, НАЗВАНИЕ, :, >, "", ԵՐԵՎԱՆԸ՝ՄԵՆՔ, ԵՆՔ, "", ՀՐԱԺԵՇՏ, ՏԱՐՐԱԿԱՆ, ԴՊՐՈՑԻՆ, 4, =, 3, ԴԱՍԱՐԱՆ, ПРОЩАНИЕ, С, НАЧАЛКОЙ, 03.05.2016, <, ОПИСАНИЕ, :, >, Ներկայացնում, ենք, 4, 3, դասարանի, տարրական, դպրոցին, հրաժեշտի, հանդեսը, տոնը, ,, որը, նվիրված, էր, մեր, քաղաքամայր, Երևանին, :, Նախագծի, հեղինակ, և, դ...","['B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'I-персона', 'I-персона', 'B-персона', 'I-персона', 'I-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'I-команда', 'B-персона', 'B-персона', 'B-персона', 'B-персон...",NaN,NaN
...,...,...,...,...
1280,"[<, НАЗВАНИЕ, :, >, 🤓, Крутизна, видео, зашкаливает, 🤓, Невероятный, Момент, 👀, Приколы, 2022, 👀, <, ОПИСАНИЕ, :, >, Подписывайтесь, на, канал, ставьте, лайки, !, Самые, свежие, и, самые, смешные, видеоролики, по, всему, миру, ., #, приколы, #, ржака, #, видеоприколы, #, юмор, #, прикол, #, смеш...","['B-персона', 'B-персона', 'B-персона', 'B-персона', 'I-серия', 'I-лига', 'B-персона', 'B-Дата', 'I-серия', 'B-персона', 'I-бренд', 'I-персона', 'B-персона', 'B-персона', 'I-серия', 'B-персона', 'B-персона', 'B-персона', 'B-персона', 'I-локация', 'B-персона', 'B-видеоигра', 'B-видеоигра', 'I-лиг...",NaN,NaN
1281,"[<, НАЗВАНИЕ, :, >, 🚘Тест, КНОПКИ, центральной, консоли, Лада, Гранта, =, Калина, <, ОПИСАНИЕ, :, >, 🔥Друзья, !, Всем, ПРИВЕТ, !.., Обязательно, подпишитесь, на, канал, !..., 🔥, 💥💥Подкиньте, копеечку, на, творчество, <, LINK,